In [11]:
import pandas as pd
import numpy as np
import sqlite3
import psycopg2
import os

In [5]:
file = '../data/flightData.csv'

In [6]:
data = pd.read_csv(file)

In [7]:
data

,Unnamed: 0,fl_date,mkt_carrier,mkt_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,dep_time,wheels_off,wheels_on,crs_arr_time,arr_time,air_time
0,0,2019-01-01,UA,5515,14869,14771,1916,1911.0,1938.0,2009.0,2028,2017.0,91.0
1,1,2019-01-01,UA,5517,11109,11292,805,930.0,1006.0,1022.0,910,1027.0,16.0
2,2,2019-01-01,UA,5518,13930,10693,1830,1833.0,1915.0,2023.0,2009,2031.0,68.0
3,3,2019-01-01,UA,5519,12266,14783,1435,1428.0,1442.0,1555.0,1627,1623.0,73.0
4,4,2019-01-01,UA,5519,13244,12266,1150,1140.0,1156.0,1331.0,1347,1343.0,95.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4995,2019-01-01,DL,5086,11193,13487,1106,1100.0,1112.0,1149.0,1211,1153.0,97.0
4996,4996,2019-01-01,DL,5087,10397,11775,1959,1954.0,2012.0,2131.0,2155,2138.0,139.0
4997,4997,2019-01-01,DL,5092,12478,11057,2055,2053.0,2128.0,2304.0,2306,2311.0,96.0
4998,4998,2019-01-01,DL,5098,11995,10397,1411,1409.0,1427.0,1523.0,1541,1530.0,56.0


In [16]:
def make_csv(query, filename):
    # check if file already exists
    if os.path.exists(filename):
        df = pd.read_csv(filename)
        return df
    # ensure all columns are displayed when viewing a pandas dataframe
    pd.set_option('display.max_columns', None)
    # Creating a connection to the database
    print("creating connection...")
    con = psycopg2.connect(database="mid_term_project",
                           user="lhl_student",
                           password="lhl_student",
                           host="lhl-data-bootcamp.crzjul5qln0e.ca-central-1.rds.amazonaws.com",
                           port="5432")
    # creating a cursor object
    cur = con.cursor()
    # running an sql query
    print("running query...")
    cur.execute(query)
    # Storing the result
    rows = cur.fetchall()
    cols = [desc[0] for desc in cur.description]
    con.close()
    # writing the csv file
    print("writing file...")
    df = pd.DataFrame(rows, columns=cols)
    df.to_csv(filename, index=False)
    print('Done')
    return df

In [18]:
quer = '''SELECT t.*
FROM (
  SELECT *, row_number() OVER(ORDER BY fl_date ASC) AS row
  FROM flights
) t
WHERE t.row % 150 = 0'''
fname = '../data/flight_data.csv'

In [21]:
make_csv(quer, fname)

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,cancelled,cancellation_code,diverted,dup,crs_elapsed_time,actual_elapsed_time,air_time,flights,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name,row
0,2018-01-01,UA,UA_CODESHARE,UA,3542,YX,N637RW,3542,13495,MSY,"New Orleans, LA",11292,DEN,"Denver, CO",1602,1812.0,130.0,11.0,1823.0,1958.0,9.0,1805,2007.0,122.0,0.0,NaN,0.0,N,183.0,175.0,155.0,1.0,1062.0,0.0,0.0,0.0,0.0,122.0,NaN,NaN,NaN,NaN,150
1,2018-01-01,UA,UA_CODESHARE,UA,3725,YX,N861RW,3725,14869,SLC,"Salt Lake City, UT",12266,IAH,"Houston, TX",1505,1459.0,-6.0,16.0,1515.0,1850.0,4.0,1910,1854.0,-16.0,0.0,NaN,0.0,N,185.0,175.0,155.0,1.0,1195.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,300
2,2018-01-01,AA,AA,AA,1675,AA,N604AW,1675,11298,DFW,"Dallas/Fort Worth, TX",10397,ATL,"Atlanta, GA",1255,1327.0,32.0,12.0,1339.0,1607.0,6.0,1557,1613.0,16.0,0.0,NaN,0.0,N,122.0,106.0,88.0,1.0,731.0,16.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,450
3,2018-01-01,AA,AA,AA,1035,AA,N314PD,1035,14679,SAN,"San Diego, CA",13930,ORD,"Chicago, IL",2259,2256.0,-3.0,21.0,2317.0,440.0,7.0,457,447.0,-10.0,0.0,NaN,0.0,N,238.0,231.0,203.0,1.0,1723.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,600
4,2018-01-01,AA,AA,AA,1412,AA,N936NN,1412,13891,ONT,"Ontario, CA",11298,DFW,"Dallas/Fort Worth, TX",615,612.0,-3.0,10.0,622.0,1042.0,13.0,1113,1055.0,-18.0,0.0,NaN,0.0,N,178.0,163.0,140.0,1.0,1188.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106178,2019-12-31,AA,AA_CODESHARE,AA,5132,OH,N530EA,5132,11278,DCA,"Washington, DC",14685,SAV,"Savannah, GA",1150,1150.0,0.0,19.0,1209.0,1335.0,6.0,1344,1341.0,-3.0,0.0,NaN,0.0,N,114.0,111.0,86.0,1.0,519.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15926850
106179,2019-12-31,AA,AA_CODESHARE,AA,5301,OH,N228PS,5301,11278,DCA,"Washington, DC",10581,BGR,"Bangor, ME",1515,1512.0,-3.0,16.0,1528.0,1655.0,6.0,1701,1701.0,0.0,0.0,NaN,0.0,N,106.0,109.0,87.0,1.0,590.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15927000
106180,2019-12-31,AA,AA_CODESHARE,AA,4760,PT,N906AE,4760,10434,AVP,"Scranton/Wilkes-Barre, PA",14100,PHL,"Philadelphia, PA",1349,1338.0,-11.0,12.0,1350.0,1413.0,7.0,1439,1420.0,-19.0,0.0,NaN,0.0,N,50.0,42.0,23.0,1.0,104.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15927150
106181,2019-12-31,AA,AA_CODESHARE,AA,4898,PT,N644AE,4898,14524,RIC,"Richmond, VA",14100,PHL,"Philadelphia, PA",618,605.0,-13.0,16.0,621.0,702.0,10.0,723,712.0,-11.0,0.0,NaN,0.0,N,65.0,67.0,41.0,1.0,198.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15927300
